In [1]:
import graphBrowser as gB

In [2]:
graphName = '<airline_flight_network>'

# Demo: construct creation function for airlines data
def createConstructQuery(origin,graphName):
    return '''PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix : <https://ontologies.semanticarts.com/raw_data#>
prefix fl: <https://ontologies.semanticarts.com/flights/>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix skos:    <http://www.w3.org/2004/02/skos/core#>
CONSTRUCT { 
  	?orig fl:hasRouteTo ?dest .
  	?orig rdfs:label ?origCode . 
  	?dest rdfs:label ?destCode . 
  	fl:hasRouteTo rdfs:label "hasRouteTo" . 
}  
WHERE { GRAPH '''+ graphName +''' {{
SELECT 
?orig ?dest ?origCode ?destCode ?miles 
    WHERE { 
        ?orig fl:terminalCode ?origCode .
        ?orig fl:hasRouteTo ?dest .
        ?dest fl:terminalCode ?destCode .
        << ?orig fl:hasRouteTo ?dest >> fl:distanceMiles ?miles .
        FILTER (?miles < 400)
        FILTER (?origCode = \'''' + origin + '''\')
  }
      }}}
'''


construct = createConstructQuery('BOS',graphName)
elements = gB.createGraph(construct,'BOS')

In [3]:
import json
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from demos import dash_reusable_components as drc

app = JupyterDash(__name__)

app.layout = html.Div([
     
    html.Div(className='a',children = [
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='circle',
        clearable=False,
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['grid', 'random', 'circle', 'cose', 'concentric']
        ]
    )]),
    drc.NamedRadioItems(
                    name='Select',
                    id='radio-option',
                    options=drc.DropdownOptionsList(
                        'Show Neighbors'
                    ),
                    value='Show Neighbors'
                ),

    html.Div(className='b',children = [
    cyto.Cytoscape(
        id='cytoscape-update-layout',
        layout={'name': 'circle'},
        style={'width': '100%', 'height': '70vh'},
        elements=elements,
        stylesheet = [
            {
            'selector': 'node',
            'style': {
                'content': 'data(label)',
                'background-color': '#58FAF4',
                }
            },
            {
            'selector': 'edge',
            'style': {
                'content': 'data(labelLabel)',
                'curve-style': 'bezier',
                'target-arrow-color': 'black',
                'target-arrow-shape': 'triangle',
                'line-color': 'black'
                }
            }
        ]
    )]),
    html.P(id='cytoscape-tapNodeData-output'),
    html.P(id='cytoscape-tapEdgeData-output')
])


##########CALLBACKS##########
@app.callback(Output('cytoscape-tapNodeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapNodeData')])
def displayTapNodeData(data):
#     print('HH')
    if data:
        return "Node selected: " + data['id']


@app.callback(Output('cytoscape-tapEdgeData-output', 'children'),
              [Input('cytoscape-update-layout', 'tapEdgeData')]
             )
def displayTapEdgeData(data):
    if data:
        return "Connection between " + data['source'].upper() + " and " + data['target'].upper() + " with edge value: " + data['label']

    
@app.callback(Output('cytoscape-update-layout', 'layout'),
              [Input('dropdown-update-layout', 'value')])
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

#expand node by neighbors
@app.callback(Output('cytoscape-update-layout', 'elements'),
              [Input('cytoscape-update-layout', 'tapNodeData')],
              [State('cytoscape-update-layout', 'elements')],
              [State('radio-option', 'value')]
             )
def generate_elements(data,e,options):

    if not data:
        return e

    nodeURI = data['id']
    if options == "Show Neighbors":
        
        if data['expanded'] == True:
            return e
 
        if data and e:
        
            #changing extended to True for the node
            for element in e:
                if data['id'] == element.get('data').get('id'):
                    element['data']['expanded'] = True
                    break
    
            # Demo: get neighboring airport creation statement for airlines data
            neighborQuery = gB.createGetNeighborsQuery(nodeURI,graphName)
            df = gB.getNeighbours(neighborQuery)
            nodes = gB.generateNodes(df,nodeURI)
            edges = gB.generateEdges(df)

            for node in nodes:
                if node not in e:
                    e.append(node)
                
            for edge in edges:
                if edge not in e:
                    e.append(edge)        
                
        return e
    elif options == "Hide Neighbors":
        
        if data['expanded'] == False:
            return e
        return gB.removeNodes(e,nodeURI)

            
            
port = '8098'
host = '127.0.0.1'

app.run_server(mode='inline',port = port, host = host)

print('App running on http://'+ host +':' + port +'/')

App running on http://127.0.0.1:8098/
